## Imports

In [1]:
import tensorflow as tf
from keras.layers.normalization import layer_normalization

In [2]:
from tensorflow import keras
from keras.layers.normalization import layer_normalization
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
# import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [3]:
train_path = r'D:\gestures\train'
test_path = r'D:\gestures\test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

Found 453 images belonging to 3 classes.
Found 453 images belonging to 3 classes.


## Building the model

In [5]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(3,activation ="softmax"))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

Epoch 1/100
46/46 [==============================] - 9s 164ms/step - loss: 3.9732 - accuracy: 0.8852 - val_loss: 0.1621 - val_accuracy: 0.9558
Epoch 2/100
46/46 [==============================] - 2s 45ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.1251 - val_accuracy: 0.9647
Epoch 3/100
46/46 [==============================] - 2s 44ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1423 - val_accuracy: 0.9625
Epoch 4/100
46/46 [==============================] - 2s 44ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1416 - val_accuracy: 0.9603


In [6]:
history2.history

{'loss': [3.973212480545044,
  0.007903863675892353,
  0.003748473944142461,
  0.002554036444053054],
 'accuracy': [0.8852097392082214, 1.0, 1.0, 1.0],
 'val_loss': [0.1620788425207138,
  0.12508046627044678,
  0.14234574139118195,
  0.14157262444496155],
 'val_accuracy': [0.9558498859405518,
  0.9646798968315125,
  0.9624723792076111,
  0.9602649211883545],
 'lr': [0.001, 0.001, 0.001, 0.0005]}

## Saving our model

In [7]:
model.save('signModelNew')

INFO:tensorflow:Assets written to: signModelNew\assets


## Metrics

In [8]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))
scores

loss of 0.00590427266433835; accuracy of 100.0%


[0.00590427266433835, 1.0]

In [12]:
imgs, labels = next(test_batches)
model = keras.models.load_model("signModelNew")
# new_model = tf.keras.models.load_model('saved_model/my_model')
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

loss of 0.0026224867906421423; accuracy of 100.0%


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

## Predictions

In [13]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three'}

predictions = model.predict(imgs, verbose=0)
predicted_labels = []
actual_labels = []

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    predicted_labels.append(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])
    actual_labels.append(word_dict[np.argmax(i)])


Predicted labels
Three
One
One
One
One
One
Two
One
Three
Two
Actual labels
Three
One
One
One
One
One
Two
One
Three
Two


In [23]:
print ("Precision", sk.metrics.precision_score(actual_labels, predicted_labels, average='macro'))
print ("Recall", sk.metrics.recall_score(actual_labels, predicted_labels, average='macro'))
print ("f1_score", sk.metrics.f1_score(actual_labels, predicted_labels, average='macro'))
print ("confusion_matrix: ")
print (sk.metrics.confusion_matrix(actual_labels, predicted_labels))

Precision 0.9583333333333334
Recall 0.8333333333333334
f1_score 0.8666666666666666
confusion_matrix: 
[[1 0 0]
 [0 1 1]
 [0 0 7]]


In [ ]:
labels

In [ ]:
predictions